
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #1


Repartir des essais de découverte précédent pour améliorer la calcul brute force au mieux.
Si la contrainte est toujours du style budget maximum, il est inutile de conserver les combinaisons dont le cout est > BUDGET.
Cela n'en devient pas pour autant de la programmation dynamique, réservée à l'algo suivant ;
la programmation dynamique évitant de recalculer des sous-arbres déja parcourus.


* Caculer simultanément à l'explosion combinatoire.
Dans les essais de découverte, j'ai dans un 1er temps explosé toutes les combinaisons possibles puis ensuite seulement calculé et limité au sous-ensemble de budget <= BUDGET.
Un calcul simultané permettrait la vérification de contrainte non atteinte et la réduction de l'ensemble des solutions possibles.

* Structure de donnée.
Pour "porter" les combinaisons une liste est envisageable avec pour chercher les données (cout, profit) de chaque action un dictionnaire semble efficace.



### 1. Préparation des données



In [1]:
import csv as csv
import re as re
import time



In [2]:
try:
    import big_o
except ModuleNotFoundError:
    print('cherche',ModuleNotFoundError)
    !pip install --upgrade pip
    !pip install big_o
    import big_o

In [3]:
# constant
#FILE = "data/p7-20-shares.csv" 
#FIELDNAMES = ['name', 'cost', 'profit_share', 'profit']

FILE = "data/dataset1_Python+P7.csv" 
FIELDNAMES = ['name', 'cost', 'profit']
BUDGET = 50000

### En complément de Big_O, mesure du temps

Si la mesure du temps d'execution est dépendante du type de PC, des processus en cours d'execution etc., elle permet une approximation de l'efficacité  des algorithme choisis complémentaire à celle de la notation big_o.

Suivant le conseil de [Marina Mele](http://www.marinamele.com/author/marina-melegmail-com) dans [7 tips to Time Python scripts and control Memory & CPU usage](https://www.marinamele.com/7-tips-to-time-python-scripts-and-control-memory-and-cpu-usage)


In [4]:
 def fn_timer(function):
        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        print ("Total time running %s: %s nanoseconds" %
               (function.__name__, str(t1-t0))
               )
        return result
    return function_timer

In [5]:
# nettoyer les données des fichiers en entrée
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

### Amélioration des données

Sur python le travail avec float est plus couteux qu'en entier.
Une solution est alors de multiplier par 100 cout, budget et profit car cela ne change pas
le résultat.


In [19]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=',', doublequote=False)
        # skip the header
        next(csv_reader)
        for idx, line in enumerate(csv_reader):
            clean_data = True

            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False

            if line[FIELDNAMES[1]] != "":
                cout = round(100 * float(clean_char(line[FIELDNAMES[1]])))
                if cout <= 0 :
                    print(f" line {idx} had null or neg cost data; dropped.")
                    cout = 0
                    clean_data = False                    
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
            if line[FIELDNAMES[2]] != "":
                gain = round(100 * float(clean_char(line[FIELDNAMES[2]])))
            else:
                print(f" line {idx} had missing profit data; dropped.")
                clean_data = False
            if (gain < 0) or (cout < 0):
                print(f" line {idx} had negative value; accepted but pls check.")
            if clean_data:
                action_dict[cle] = (cout, gain)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")            


 line 5 had null or neg cost data; dropped.
 line 9 had null or neg cost data; dropped.
 line 32 had null or neg cost data; dropped.
 line 34 had null or neg cost data; dropped.
 line 95 had null or neg cost data; dropped.
 line 110 had null or neg cost data; dropped.
 line 123 had null or neg cost data; dropped.
 line 132 had null or neg cost data; dropped.
 line 170 had null or neg cost data; dropped.
 line 186 had null or neg cost data; dropped.
 line 234 had null or neg cost data; dropped.
 line 252 had null or neg cost data; dropped.
 line 286 had null or neg cost data; dropped.
 line 314 had null or neg cost data; dropped.
 line 412 had null or neg cost data; dropped.
 line 423 had null or neg cost data; dropped.
 line 445 had null or neg cost data; dropped.
 line 448 had null or neg cost data; dropped.
 line 472 had null or neg cost data; dropped.
 line 515 had null or neg cost data; dropped.
 line 531 had null or neg cost data; dropped.
 line 532 had null or neg cost data; drop

In [1]:
#action_dict
                

### 2. Calcul brute force.

$$
{n \choose k}={\frac {n!}{k!(n-k)!}}.
$$

### 2.1 Estimation simple du nombre de calcul maximum.

entendre : Non réduit à la contrainte BUDGET.

In [22]:
def number_combi(n: int, k: int) -> int:
    def factorielle(x:int) -> int:
        if x <=1:
            return 1
        else:
            return (x * factorielle(x-1))
    top = factorielle(n)
    bot = factorielle(k) * factorielle(n-k)
    if bot > 0:
        return top/bot
    else:
        raise Error

In [24]:
denombrement = True
if denombrement:
    N = len(action_dict.keys())
    number_of_combi = 0
    for K in range(N):
        number_of_combi += number_combi(N, K)
    print(number_of_combi,' combinaisons pour un échantillon de ', N)   

6.090821257124996e+287  combinaisons pour un échantillon de  956



Nous sommes prévenu, il y a environ 6.090821257124996e+287 de combinaison d'un portefeuille de 956 actions!
Le calcul des combinaisons est impossible en Force Brute. 
  
